In [1]:
from pyspark.sql import SparkSession
from py4j.java_gateway import java_import

spark = (
    SparkSession.builder
        .appName("Delta S3 MRI Notebook")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

        # MinIO S3 connection
        .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        # Warehouse root
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")

        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
        .config("spark.sql.catalogImplementation", "hive")

        .getOrCreate()
)

print("Spark session started:", spark)

# --------------------------
# MRI paths
# --------------------------
mri_root_path = "s3a://warehouse/fs/mri"
mri_archive_root = "s3a://warehouse/fs/mri_archive"
mri_table = "hyb_bronze.mri_raw"

# Prepare Hadoop FS
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
java_import(sc._jvm, "java.net.URI")

fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
    sc._jvm.java.net.URI(mri_root_path), 
    hadoop_conf
)

mri_root = sc._jvm.org.apache.hadoop.fs.Path(mri_root_path)

print("MRI root path ready:", mri_root_path)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/10 06:18:21 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/10 06:18:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-55c86b1c-8b65-4b03-abc0-ac84f8c24d01;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	f

Spark session started: <pyspark.sql.session.SparkSession object at 0x7110cd7556c0>


25/12/10 06:18:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


MRI root path ready: s3a://warehouse/fs/mri


In [5]:
import os

local_root = "/home/ubuntu2020/Downloads/k57fr854j2-2/MRI_Data/01_MRI_Data/0001"
s3_root = "s3a://warehouse/fs/mri/0001"

print("Uploading MRI dataset to MinIO...")
uploaded = 0

for root, dirs, files in os.walk(local_root):
    for file in files:
        local_file = os.path.join(root, file)

        # Compute relative folder structure
        relative = os.path.relpath(local_file, local_root)
        s3_path = f"{s3_root}/{relative}"

        # Read binary file
        with open(local_file, "rb") as f:
            binary = f.read()

        # Write binary data to S3
        (
            spark.sparkContext
                .parallelize([binary])
                .saveAsTextFile(s3_path)
        )

        print("Uploaded:", s3_path)
        uploaded += 1

print("\nUpload complete!")
print("Total MRI files uploaded:", uploaded)


Uploading MRI dataset to MinIO...


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T1_TSE_TRA_0005/T1_TSE_TRA__0001_005.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T1_TSE_TRA_0005/T1_TSE_TRA__0001_006.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T1_TSE_TRA_0005/T1_TSE_TRA__0001_003.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T1_TSE_TRA_0005/T1_TSE_TRA__0001_004.ima


...

Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_004.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_005.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_007.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_006.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_011.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_008.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_015.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_003.ima
Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_002.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_013.ima


Uploaded: s3a://warehouse/fs/mri/0001/L-SPINE_LSS_20160309_091629_240000/T2_TSE_SAG_384_0002/T2_TSE_SAG__0001_012.ima

Upload complete!
Total MRI files uploaded: 65
